In [125]:
import pandas as pd
import wbgapi as wb
import numpy as np
from sqlalchemy import create_engine
import psycopg2

### Needed Series from WDI Database 
- EN.ATM.CO2E.KT - CO2 emissions (kt)
- EN.ATM.CO2E.PC - CO2 emissions (metric tons per capita)
- EN.ATM.CO2E.PP.GD.KD	- CO2 emissions (kg per 2017 PPP dollar of GDP)
- EG.USE.PCAP.KG.OE	 - Energy Use (kg of oil equivalent per capita)
- EG.FEC.RNEW.ZS - Renewable energy consumption (% of total final energy consumption)
- AG.YLD.CREL.KG - Cereal yield
- EG.ELC.ACCS.ZS - Access to Electricity (% of population)
- AG.LND.FRST.ZS - Forest area (% of land area)
- NY.GDP.MKTP.KD.ZG - GDP growth (annual %)
- NY.GDP.PCAP.CD - GDP per capita (current USD)
- NY.GNP.PCAP.CD - GNI per capita, Atlas method (current USD)
- SP.POP.TOTL - Total Population
- SP.POP.GROW - Population growth (annual %)
- SP.URB.GROW - Urban population growth (annual %)
- SP.URB.TOTL - Urban population
- SP.URB.TOTL.IN.ZS	- Urban population (% of total population)

## Extract emissions data from World Bank database via API (wbgapi)
### Ensure conda environment has wbgapi installed (pip install wbgapi)

In [126]:
series_list = ['EN.ATM.CO2E.KT',
               'EN.ATM.CO2E.PC',
               'EN.ATM.CO2E.PP.GD.KD',
               'EG.USE.PCAP.KG.OE',
               'EG.FEC.RNEW.ZS',
               'AG.YLD.CREL.KG',
               'EG.ELC.ACCS.ZS',
               'AG.LND.FRST.ZS',
               'NY.GDP.MKTP.KD.ZG',
               'NY.GDP.PCAP.CD',
               'NY.GNP.PCAP.CD',
               'SP.POP.TOTL',
               'SP.POP.GROW',
               'SP.URB.GROW',
               'SP.URB.TOTL',
               'SP.URB.TOTL.IN.ZS']

In [127]:
# Using wbgapi to extract World Bank data as Pandas data frame
raw_df = wb.data.DataFrame(series_list, time=range(1990, 2018), numericTimeKeys=True, labels=True, columns='series').reset_index()
raw_df.head()

,economy,time,Country,Time,AG.LND.FRST.ZS,AG.YLD.CREL.KG,EG.ELC.ACCS.ZS,EG.FEC.RNEW.ZS,EG.USE.PCAP.KG.OE,EN.ATM.CO2E.KT,EN.ATM.CO2E.PC,EN.ATM.CO2E.PP.GD.KD,NY.GDP.MKTP.KD.ZG,NY.GDP.PCAP.CD,NY.GNP.PCAP.CD,SP.POP.GROW,SP.POP.TOTL,SP.URB.GROW,SP.URB.TOTL,SP.URB.TOTL.IN.ZS
0,ZWE,2017,Zimbabwe,2017,45.45,1202.70,44.18,82.46,NaN,10340.00,0.70,0.30,4.08,1192.11,1170.00,2.04,14751101.00,1.86,4755312.00,32.24
1,ZWE,2016,Zimbabwe,2016,45.57,435.10,42.56,81.90,NaN,11020.00,0.76,0.33,0.90,1421.79,1200.00,2.08,14452704.00,1.81,4667645.00,32.30
2,ZWE,2015,Zimbabwe,2015,45.69,557.50,33.70,80.82,NaN,12430.00,0.88,0.38,2.02,1410.33,1220.00,2.14,14154937.00,1.77,4584076.00,32.38
3,ZWE,2014,Zimbabwe,2014,45.81,831.40,32.30,80.27,NaN,12080.00,0.87,0.38,1.48,1407.03,1210.00,2.19,13855753.00,1.73,4503674.00,32.50
4,ZWE,2013,Zimbabwe,2013,45.93,668.50,40.50,78.87,832.57,12280.00,0.91,0.39,3.20,1408.37,1200.00,2.16,13555422.00,1.61,4426387.00,32.65


In [128]:
# Rows and columns of data set
raw_df.shape

(7448, 20)

In [129]:
# # Datatypes of columns
raw_df.dtypes

economy                  object
time                      int64
Country                  object
Time                     object
AG.LND.FRST.ZS          float64
AG.YLD.CREL.KG          float64
EG.ELC.ACCS.ZS          float64
EG.FEC.RNEW.ZS          float64
EG.USE.PCAP.KG.OE       float64
EN.ATM.CO2E.KT          float64
EN.ATM.CO2E.PC          float64
EN.ATM.CO2E.PP.GD.KD    float64
NY.GDP.MKTP.KD.ZG       float64
NY.GDP.PCAP.CD          float64
NY.GNP.PCAP.CD          float64
SP.POP.GROW             float64
SP.POP.TOTL             float64
SP.URB.GROW             float64
SP.URB.TOTL             float64
SP.URB.TOTL.IN.ZS       float64
dtype: object

In [130]:
# Descriptive statistics
raw_df.describe()

,time,AG.LND.FRST.ZS,AG.YLD.CREL.KG,EG.ELC.ACCS.ZS,EG.FEC.RNEW.ZS,EG.USE.PCAP.KG.OE,EN.ATM.CO2E.KT,EN.ATM.CO2E.PC,EN.ATM.CO2E.PP.GD.KD,NY.GDP.MKTP.KD.ZG,NY.GDP.PCAP.CD,NY.GNP.PCAP.CD,SP.POP.GROW,SP.POP.TOTL,SP.URB.GROW,SP.URB.TOTL,SP.URB.TOTL.IN.ZS
count,7448.00,7117.00,6175.00,6305.00,7089.00,4751.00,6669.00,6669.00,6206.00,6782.00,6944.00,6393.00,7418.00,7420.00,7363.00,7364.00,7364.00
mean,2003.50,32.63,2894.70,79.93,31.16,2270.16,992709.46,4.21,0.27,3.58,11171.75,9479.31,1.48,258308400.43,2.23,121029449.38,55.27
std,8.08,23.39,2336.05,29.46,30.12,2669.53,3204384.87,5.23,0.22,5.79,18975.24,15075.52,1.65,819608141.63,2.09,380606489.23,23.53
min,1990.00,0.00,0.10,0.53,0.00,9.58,0.00,0.00,0.00,-64.05,22.85,40.00,-27.72,9182.00,-27.71,3733.00,5.42
25%,1996.75,12.51,1404.05,65.93,4.79,603.05,2230.00,0.64,0.14,1.55,1007.13,910.00,0.55,1330466.00,0.77,651572.25,35.30
50%,2003.50,30.86,2388.70,98.30,20.95,1238.11,23740.00,2.42,0.22,3.71,3322.03,3020.00,1.41,8483160.50,2.20,4041233.00,54.16
75%,2010.25,47.62,3796.25,100.00,54.76,3025.74,246490.00,6.22,0.33,5.90,13096.14,10400.00,2.42,55932344.25,3.50,31827577.25,74.31
max,2017.00,98.57,36761.90,100.00,98.34,21420.63,33514537.91,47.65,2.09,149.97,203266.91,122130.00,19.36,7578157615.00,31.14,4147418821.00,100.00


## Data clean and prep

In [131]:
# Assign original dataframe to another that can be modified
emissions_df = raw_df
emissions_df.sample(5)

,economy,time,Country,Time,AG.LND.FRST.ZS,AG.YLD.CREL.KG,EG.ELC.ACCS.ZS,EG.FEC.RNEW.ZS,EG.USE.PCAP.KG.OE,EN.ATM.CO2E.KT,EN.ATM.CO2E.PC,EN.ATM.CO2E.PP.GD.KD,NY.GDP.MKTP.KD.ZG,NY.GDP.PCAP.CD,NY.GNP.PCAP.CD,SP.POP.GROW,SP.POP.TOTL,SP.URB.GROW,SP.URB.TOTL,SP.URB.TOTL.IN.ZS
5093,CAN,1992,Canada,1992,38.84,2460.30,100.00,21.93,7601.69,426370.00,15.03,NaN,NaN,20879.85,NaN,1.18,28371264.00,1.53,21813814.00,76.89
2025,NGA,2008,Nigeria,2008,25.90,1598.40,50.30,86.25,742.81,86930.00,0.57,0.13,6.76,2227.79,1950.00,2.72,152382506.00,4.86,63546553.00,41.70
4987,TCD,2014,Chad,2014,3.99,855.70,8.41,76.86,NaN,2260.00,0.16,0.09,6.90,1017.79,970.00,3.57,13697126.00,4.13,3064321.00,22.37
3745,GUY,1996,Guyana,1996,94.38,3958.10,71.99,38.61,NaN,1580.00,2.10,0.29,7.96,936.08,840.00,0.25,753571.00,-0.06,218897.00,29.05
6434,OED,1995,OECD members,1995,32.38,3829.88,99.32,7.58,4259.97,11859670.00,10.26,0.33,2.66,22207.04,21639.47,0.81,1155551702.00,1.18,859678033.00,74.40


In [132]:
emissions_df.isnull().sum()

economy                    0
time                       0
Country                    0
Time                       0
AG.LND.FRST.ZS           331
AG.YLD.CREL.KG          1273
EG.ELC.ACCS.ZS          1143
EG.FEC.RNEW.ZS           359
EG.USE.PCAP.KG.OE       2697
EN.ATM.CO2E.KT           779
EN.ATM.CO2E.PC           779
EN.ATM.CO2E.PP.GD.KD    1242
NY.GDP.MKTP.KD.ZG        666
NY.GDP.PCAP.CD           504
NY.GNP.PCAP.CD          1055
SP.POP.GROW               30
SP.POP.TOTL               28
SP.URB.GROW               85
SP.URB.TOTL               84
SP.URB.TOTL.IN.ZS         84
dtype: int64

In [133]:
#Remove Time column as it is not neccessary column
emissions_df.drop(columns = ['Time'], axis = 1, inplace = True)

In [134]:
# Rename the columns headers to meaningful names
column_names = {'AG.LND.FRST.ZS':'forest_area_percent',
             'AG.YLD.CREL.KG':'cereal_yield',
             'EG.ELC.ACCS.ZS':'electricity_access_percent',
             'EG.FEC.RNEW.ZS':'renew_energy_percent',
             'EG.USE.PCAP.KG.OE':'energy_use_per_capita',
             'EN.ATM.CO2E.KT':'emissions_total',
             'EN.ATM.CO2E.PC':'emissions_per_capita',
             'EN.ATM.CO2E.PP.GD.KD':'emissions_per_gdp',
             'NY.GDP.MKTP.KD.ZG':'gdp_growth_percent',
             'NY.GDP.PCAP.CD':'gdp_per_capita',
             'NY.GNP.PCAP.CD':'gni_per_capita',
             'SP.POP.GROW':'pop_growth_percent',
             'SP.POP.TOTL':'pop_total',
             'SP.URB.GROW':'urb_pop_growth_percent',
             'SP.URB.TOTL':'urban_pop_total',
             'SP.URB.TOTL.IN.ZS':'urban_pop_percent',
             'economy':'country_code',
             'time':'year',
             'Country':'country_name'    
            }

In [135]:
emissions_df = emissions_df.rename(columns=column_names)
emissions_df.sample(10)

,country_code,year,country_name,forest_area_percent,cereal_yield,electricity_access_percent,renew_energy_percent,energy_use_per_capita,emissions_total,emissions_per_capita,emissions_per_gdp,gdp_growth_percent,gdp_per_capita,gni_per_capita,pop_growth_percent,pop_total,urb_pop_growth_percent,urban_pop_total,urban_pop_percent
4387,GNQ,1998,Equatorial Guinea,93.84,NaN,NaN,68.02,NaN,1970.00,3.13,0.43,23.77,588.64,620.00,3.91,629734.00,7.76,287259.00,45.62
1119,ZAF,1990,South Africa,14.96,1877.30,NaN,16.63,2280.88,247660.00,6.21,0.60,-0.32,3160.88,2890.00,3.08,39877570.00,4.11,20751091.00,52.04
5958,AND,1995,Andorra,34.04,NaN,100.00,14.48,NaN,430.00,6.83,NaN,2.76,18731.55,NaN,3.05,62928.00,2.81,58939.00,93.66
2156,NLD,2017,Netherlands,10.89,8794.50,100.00,6.36,NaN,155759.99,9.09,0.17,2.91,48675.22,46270.00,0.59,17131296.00,1.08,15602670.00,91.08
6440,INX,2017,Not classified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
311,GBR,2014,United Kingdom,12.96,7696.50,100.00,7.36,2777.31,415600.01,6.43,0.15,3.20,47447.59,44590.00,0.74,64602298.00,1.06,53209683.00,82.36
5057,CYM,2000,Cayman Islands,53.88,NaN,100.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.50,39658.00,3.50,39658.00,100.00
6106,UMC,2015,Upper middle income,37.49,4365.22,98.89,13.67,NaN,14889729.52,6.16,0.41,4.31,8132.29,8432.22,0.81,2417959696.00,2.23,1532338785.00,63.42
6426,OED,2003,OECD members,32.45,4251.16,99.60,7.89,4425.43,13020410.14,10.61,0.29,2.04,26030.55,24696.05,0.74,1227425785.00,1.25,941906450.00,76.74
5215,BFA,2010,Burkina Faso,24.55,1062.70,13.10,81.45,NaN,2290.00,0.14,0.09,8.45,627.27,630.00,2.94,16116845.00,5.21,3970062.00,24.63


In [136]:
#swap year and country_name columns
col_list = list(emissions_df.columns)
x, y = col_list.index('year'), col_list.index('country_name')
col_list[y], col_list[x] = col_list[x], col_list[y]
emissions_df = emissions_df[col_list]
list(emissions_df.columns)

['country_code',
 'country_name',
 'year',
 'forest_area_percent',
 'cereal_yield',
 'electricity_access_percent',
 'renew_energy_percent',
 'energy_use_per_capita',
 'emissions_total',
 'emissions_per_capita',
 'emissions_per_gdp',
 'gdp_growth_percent',
 'gdp_per_capita',
 'gni_per_capita',
 'pop_growth_percent',
 'pop_total',
 'urb_pop_growth_percent',
 'urban_pop_total',
 'urban_pop_percent']

In [137]:
# Official countries list from https://unstats.un.org/unsd/methodology/m49/
countries = ["AFG","ALA","ALB","DZA","ASM","AND","AGO","AIA","ATA","ATG","ARG","ARM","ABW","AUS","AUT","AZE","BHS","BHR","BGD","BRB","BLR","BEL","BLZ","BEN","BMU","BTN","BOL","BES","BIH","BWA","BVT","BRA","IOT","VGB","BRN","BGR","BFA","BDI","CPV","KHM","CMR","CAN","CYM","CAF","TCD","CHL","CHN","HKG","MAC","CXR","CCK","COL","COM","COG","COK","CRI","CIV","HRV","CUB","CUW","CYP","CZE","PRK","COD","DNK","DJI","DMA","DOM","ECU","EGY","SLV","GNQ","ERI","EST","SWZ","ETH","FLK","FRO","FJI","FIN","FRA","GUF","PYF","ATF","GAB","GMB","GEO","DEU","GHA","GIB","GRC","GRL","GRD","GLP","GUM","GTM","GGY","GIN","GNB","GUY","HTI","HMD","VAT","HND","HUN","ISL","IND","IDN","IRN","IRQ","IRL","IMN","ISR","ITA","JAM","JPN","JEY","JOR","KAZ","KEN","KIR","KWT","KGZ","LAO","LVA","LBN","LSO","LBR","LBY","LIE","LTU","LUX","MDG","MWI","MYS","MDV","MLI","MLT","MHL","MTQ","MRT","MUS","MYT","MEX","FSM","MCO","MNG","MNE","MSR","MAR","MOZ","MMR","NAM","NRU","NPL","NLD","NCL","NZL","NIC","NER","NGA","NIU","NFK","MKD","MNP","NOR","OMN","PAK","PLW","PAN","PNG","PRY","PER","PHL","PCN","POL","PRT","PRI","QAT","KOR","MDA","REU","ROU","RUS","RWA","BLM","SHN","KNA","LCA","MAF","SPM","VCT","WSM","SMR","STP","","SAU","SEN","SRB","SYC","SLE","SGP","SXM","SVK","SVN","SLB","SOM","ZAF","SGS","SSD","ESP","LKA","PSE","SDN","SUR","SJM","SWE","CHE","SYR","TJK","THA","TLS","TGO","TKL","TON","TTO","TUN","TUR","TKM","TCA","TUV","UGA","UKR","ARE","GBR","TZA","UMI","USA","VIR","URY","UZB","VUT","VEN","VNM","WLF","ESH","YEM","ZMB","ZWE"]
len(countries) # 249 countries

249

In [138]:
#List of countries which are not in countries official list
countries_to_drop = emissions_df.loc[~emissions_df["country_code"].isin(countries)]
countries_to_drop['country_name'].unique()
# After filtering based on official list of countries we have data for 150 countries (removes 852 records)

array(['Kosovo', 'Channel Islands', 'World', 'Upper middle income',
       'Sub-Saharan Africa (IDA & IBRD countries)',
       'Sub-Saharan Africa (excluding high income)', 'Sub-Saharan Africa',
       'South Asia (IDA & IBRD)', 'South Asia', 'Small states',
       'Pre-demographic dividend', 'Post-demographic dividend',
       'Pacific island small states', 'Other small states',
       'OECD members', 'Not classified', 'North America', 'Middle income',
       'Middle East & North Africa (IDA & IBRD countries)',
       'Middle East & North Africa (excluding high income)',
       'Middle East & North Africa', 'Lower middle income', 'Low income',
       'Low & middle income',
       'Least developed countries: UN classification',
       'Latin America & the Caribbean (IDA & IBRD countries)',
       'Latin America & Caribbean (excluding high income)',
       'Latin America & Caribbean', 'Late-demographic dividend',
       'IDA total', 'IDA only', 'IDA blend', 'IDA & IBRD total',
       'I

In [139]:
#Select countries which are in countries official list
emissions_df = emissions_df.loc[emissions_df["country_code"].isin(countries)]
emissions_df

,country_code,country_name,year,forest_area_percent,cereal_yield,electricity_access_percent,renew_energy_percent,energy_use_per_capita,emissions_total,emissions_per_capita,emissions_per_gdp,gdp_growth_percent,gdp_per_capita,gni_per_capita,pop_growth_percent,pop_total,urb_pop_growth_percent,urban_pop_total,urban_pop_percent
0,ZWE,Zimbabwe,2017,45.45,1202.70,44.18,82.46,NaN,10340.00,0.70,0.30,4.08,1192.11,1170.00,2.04,14751101.00,1.86,4755312.00,32.24
1,ZWE,Zimbabwe,2016,45.57,435.10,42.56,81.90,NaN,11020.00,0.76,0.33,0.90,1421.79,1200.00,2.08,14452704.00,1.81,4667645.00,32.30
2,ZWE,Zimbabwe,2015,45.69,557.50,33.70,80.82,NaN,12430.00,0.88,0.38,2.02,1410.33,1220.00,2.14,14154937.00,1.77,4584076.00,32.38
3,ZWE,Zimbabwe,2014,45.81,831.40,32.30,80.27,NaN,12080.00,0.87,0.38,1.48,1407.03,1210.00,2.19,13855753.00,1.73,4503674.00,32.50
4,ZWE,Zimbabwe,2013,45.93,668.50,40.50,78.87,832.57,12280.00,0.91,0.39,3.20,1408.37,1200.00,2.16,13555422.00,1.61,4426387.00,32.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6071,AFG,Afghanistan,1994,1.85,1140.40,NaN,32.80,NaN,1290.00,0.08,NaN,NaN,NaN,NaN,9.86,15455555.00,10.28,3328199.00,21.53
6072,AFG,Afghanistan,1993,1.85,1132.90,NaN,30.59,NaN,1340.00,0.10,NaN,NaN,NaN,NaN,14.96,14003760.00,15.38,3002966.00,21.44
6073,AFG,Afghanistan,1992,1.85,1097.80,NaN,26.52,NaN,1390.00,0.12,NaN,NaN,NaN,NaN,11.52,12057433.00,11.94,2574865.00,21.36
6074,AFG,Afghanistan,1991,1.85,1160.40,NaN,17.04,NaN,2230.00,0.21,NaN,NaN,NaN,NaN,0.47,10745167.00,0.89,2285067.00,21.27


In [145]:
#make a copy of df to clean more
clean_df = emissions_df
clean_df = clean_df.dropna(thresh = 16)
len(clean_df)

4991

In [146]:
clean_df.isnull().sum()

country_code                     0
country_name                     0
year                             0
forest_area_percent             29
cereal_yield                   413
electricity_access_percent     631
renew_energy_percent            20
energy_use_per_capita         1547
emissions_total                  0
emissions_per_capita             0
emissions_per_gdp              165
gdp_growth_percent              43
gdp_per_capita                   3
gni_per_capita                 158
pop_growth_percent               1
pop_total                        0
urb_pop_growth_percent           0
urban_pop_total                  0
urban_pop_percent                0
dtype: int64

In [147]:
#define an array with the unique year values
years_count_missing = dict.fromkeys(clean_df['year'].unique(), 0)
for ind, row in clean_df.iterrows():
    years_count_missing[row['year']] += row.isnull().sum()

# sort the years by missing values
years_missing_sorted = dict(sorted(years_count_missing.items(), key=lambda item: item[1]))

# print the missing values for each year
print("missing values by year:")
for key, val in years_missing_sorted.items():
    print(key, ":", val)

missing values by year:
2007 : 45
2006 : 46
2004 : 49
2005 : 51
2012 : 68
2011 : 70
2010 : 70
2009 : 70
2008 : 71
2000 : 73
2013 : 74
2003 : 75
2002 : 77
2014 : 78
2001 : 80
1999 : 113
1998 : 117
1997 : 125
1996 : 140
1994 : 142
1993 : 149
1995 : 151
1990 : 159
1992 : 165
2015 : 169
1991 : 181
2017 : 201
2016 : 201


In [148]:
print("number of missing values in the whole dataset before filtering the years:")
print(clean_df.isnull().sum().sum())
print("number of rows before filtering the years:")
print(clean_df.shape[0])

# filter only rows for years between 1995 and 2015 (having less missing values)
clean_df = clean_df[(clean_df['year'] >= 1995) & (clean_df['year'] <= 2015)]

print("number of missing values in the whole dataset after filtering the years:")
print(clean_df.isnull().sum().sum())
print("number of rows after filtering the years:")
print(clean_df.shape[0])

number of missing values in the whole dataset before filtering the years:
3010
number of rows before filtering the years:
4991
number of missing values in the whole dataset after filtering the years:
1812
number of rows after filtering the years:
3834


In [149]:
clean_df.isnull().sum()

country_code                    0
country_name                    0
year                            0
forest_area_percent            16
cereal_yield                  315
electricity_access_percent    243
renew_energy_percent           10
energy_use_per_capita         991
emissions_total                 0
emissions_per_capita            0
emissions_per_gdp             125
gdp_growth_percent             26
gdp_per_capita                  0
gni_per_capita                 86
pop_growth_percent              0
pop_total                       0
urb_pop_growth_percent          0
urban_pop_total                 0
urban_pop_percent               0
dtype: int64

In [150]:
pd.set_option("display.float_format", lambda x : f"{x:.2f}")
emissions_df = clean_df.replace(np.nan, 0.0)
emissions_df

,country_code,country_name,year,forest_area_percent,cereal_yield,electricity_access_percent,renew_energy_percent,energy_use_per_capita,emissions_total,emissions_per_capita,emissions_per_gdp,gdp_growth_percent,gdp_per_capita,gni_per_capita,pop_growth_percent,pop_total,urb_pop_growth_percent,urban_pop_total,urban_pop_percent
2,ZWE,Zimbabwe,2015,45.69,557.50,33.70,80.82,0.00,12430.00,0.88,0.38,2.02,1410.33,1220.00,2.14,14154937.00,1.77,4584076.00,32.38
3,ZWE,Zimbabwe,2014,45.81,831.40,32.30,80.27,0.00,12080.00,0.87,0.38,1.48,1407.03,1210.00,2.19,13855753.00,1.73,4503674.00,32.50
4,ZWE,Zimbabwe,2013,45.93,668.50,40.50,78.87,832.57,12280.00,0.91,0.39,3.20,1408.37,1200.00,2.16,13555422.00,1.61,4426387.00,32.65
5,ZWE,Zimbabwe,2012,46.05,695.70,44.00,77.50,814.91,12010.00,0.91,0.39,15.74,1290.19,1120.00,1.82,13265331.00,1.27,4355539.00,32.83
6,ZWE,Zimbabwe,2011,46.17,587.40,36.90,79.27,787.03,11410.00,0.88,0.43,14.62,1082.62,950.00,1.44,13025785.00,0.89,4300463.00,33.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6059,AFG,Afghanistan,2006,1.85,1551.70,30.72,31.89,0.00,1760.00,0.07,0.05,5.36,274.00,0.00,4.14,25442944.00,5.03,5828215.00,22.91
6060,AFG,Afghanistan,2005,1.85,1790.40,25.39,33.88,0.00,1550.00,0.06,0.05,11.23,255.06,0.00,3.58,24411191.00,4.47,5542073.00,22.70
6061,AFG,Afghanistan,2004,1.85,1334.80,20.06,44.24,0.00,1030.00,0.04,0.03,1.41,221.66,0.00,3.93,23553551.00,4.59,5299549.00,22.50
6062,AFG,Afghanistan,2003,1.85,1458.00,14.74,36.66,0.00,1220.00,0.05,0.04,8.83,200.46,0.00,7.54,22645130.00,7.95,5061866.00,22.35


In [152]:
#Save data into csv file
emissions_df.to_csv('../Resources/emissions.csv',index= False)